# 한빛 아카데미 - 새로나온책 10page

* https://www.hanbit.co.kr/academy/books/new_book_list.html 
* 제목 / 저자  / 번역 /  출간일 / 쪽수 / 판매가

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
url = base_url + sub_url + '?page=' + str(1)
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

# 1. 찾으려고 하는 목록 데이터

In [3]:
lis = soup.select('li.sub_book_list')
len(lis)

20

## 2. 하나의 데이터로 원하는 정보를 추출

In [16]:
li = lis[1]

In [17]:
#서브페이지 링크 추출
href = li.find('a')['href']
href

'/academy/books/book_view.html?p_code=B8870109394'

In [18]:
# sub page 로 이동해서 원하는 정보를 추출
sub_req = requests.get(base_url + href)
sub_soup = BeautifulSoup(sub_req.text, 'html.parser')
# req = requests.get(url)
# soup = BeautifulSoup(req.text, 'html.parser')
sub_soup

<!DOCTYPE html>

<html lang="ko">
<head>
<!--[if lte IE 8]>
<script>
  location.replace('/support/explorer_upgrade.html');
</script>
<![endif]-->
<meta charset="utf-8"/>
<title>IT CookBook, 컴퓨터 비전과 딥러닝</title>
<link href="https://www.hanbit.co.kr/images/common/hanbit.ico" rel="shortcut icon"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="website" property="og:type"/>
<meta content="IT CookBook, 컴퓨터 비전과 딥러닝" property="og:title"/>
<meta content="규칙 기반의 고전 컴퓨터 비전을 지원하는 OpenCV와 데이터 중심의 딥러닝 컴퓨터 비전을 지원하는 텐서플로를 활용한 85개 파이썬 프로그램으로 컴퓨터 비전을 균형 있게 배울 수 있습니다." property="og:description"/>
<meta content="https://www.hanbit.co.kr/data/books/B8870109394_m.jpg" property="og:image"/>
<meta content="https://www.hanbit.co.kr/academy/books/book_view.html?p_code=B8870109394" property="og:url"/>
<link href="https://www.hanbit.co.kr/academy/books/book_view.html?p_code=B8870109394" rel="canonical">
<meta content="컴퓨터 비전,딥러닝,파이썬,OpenCV,텐서플로" name="keywords"/>
<meta content="규칙 기반의 고전 컴퓨

In [19]:
# title
title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
title

'IT CookBook, 컴퓨터 비전과 딥러닝'

In [20]:
info_list = sub_soup.select('.info_list > li ')
info_list

[<li><strong>저자 : </strong><span>오일석 </span></li>,
 <li><strong>출간 : </strong><span>2023-01-05</span></li>,
 <li><strong>페이지 : </strong><span>664 쪽</span></li>,
 <li><strong>ISBN : </strong><span>9791156645481</span></li>,
 <li><strong>물류코드 :</strong><span>4548</span></li>]

In [22]:
dict(info_list)

{<strong>저자 : </strong>: <span>오일석 </span>,
 <strong>출간 : </strong>: <span>2023-01-05</span>,
 <strong>페이지 : </strong>: <span>664 쪽</span>,
 <strong>ISBN : </strong>: <span>9791156645481</span>,
 <strong>물류코드 :</strong>: <span>4548</span>}

In [23]:
info_dict = {}
for li in info_list:
    key = li.find('strong').get_text().split(' ')[0]
    val = li.find('span').get_text().strip()
    info_dict[key] = val
info_dict

{'저자': '오일석',
 '출간': '2023-01-05',
 '페이지': '664 쪽',
 'ISBN': '9791156645481',
 '물류코드': '4548'}

In [24]:
author = info_dict['저자']
translator = info_dict['번역']
date = info_dict['출간']
page = info_dict['페이지']
author,translator, date, page

KeyError: '번역'

In [25]:
try: 
    translator = info_dict['번역']
except:
    translator = '-'

In [26]:
author,translator, date, page

('오일석', '-', '2023-01-10', '592 쪽')

In [28]:
page = int(info_dict['페이지'].split(' ')[0].replace(',', ''))
page

664

In [29]:
price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',', ''))
price

39000

## 한페이지 정보 다 가져오기

In [31]:
lines = [] 
for li in lis:
    href = li.find('a')['href']
    sub_req = requests.get(base_url + href)
    sub_soup = BeautifulSoup(sub_req.text, 'html.parser')
    
    title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
    info_list = sub_soup.select('.info_list > li ')
    info_dict = {}
    for li in info_list:
        key = li.find('strong').get_text().split(' ')[0]
        val = li.find('span').get_text().strip()
        info_dict[key] = val
    author = info_dict['저자']        
    try: 
        translator = info_dict['번역']
    except:
        translator = '-' 
    date = info_dict['출간']
    page = int(info_dict['페이지'].split(' ')[0].replace(',', ''))
    price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',', ''))
    lines.append([title, author, translator, date, page, price])

## 데이터 프레임으로 변환

In [33]:
df = pd.DataFrame(lines, columns=['제목', '저자','번역', '출간일', '쪽수', '판매가'])
df.head()

,제목,저자,번역,출간일,쪽수,판매가
0,"STEM CookBook, 전기 및 하이브리드 자동차(3판)",Iqbal Husain,"임원식 , 김정윤 , 김기찬 , 강동우",2023-01-10,592,35000
1,"IT CookBook, 컴퓨터 비전과 딥러닝",오일석,-,2023-01-05,664,39000
2,리더십 : 이론과 실제 (9판),Peter G. Northouse,김남현,2023-01-02,660,36000
3,"IT CookBook, JAVA 마스터",송미영,-,2023-01-01,732,30000
4,"IT CookBook, 데이터 과학을 위한 파이썬 프로그래밍(2판)",최성철,-,2023-01-01,540,30000


## 10페이지 정보 가져오기

In [34]:
lines = [] 
for page in tqdm(range(1, 11)):
    url = base_url + sub_url + '?page=' + str(page)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    lis = soup.select('li.sub_book_list')
    
    for li in lis:
        href = li.find('a')['href']
        sub_req = requests.get(base_url + href)
        sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

        title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
        info_list = sub_soup.select('.info_list > li ')
        info_dict = {}
        for li in info_list:
            key = li.find('strong').get_text().split(' ')[0]
            val = li.find('span').get_text().strip()
            info_dict[key] = val
        author = info_dict['저자']        
        try: 
            translator = info_dict['번역']
        except:
            translator = '-' 
        date = info_dict['출간']
        page = int(info_dict['페이지'].split(' ')[0].replace(',', ''))
        price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',', ''))
        lines.append([title, author, translator, date, page, price])

  0%|          | 0/10 [00:00<?, ?it/s]

In [35]:
df = pd.DataFrame(lines, columns=['제목', '저자','번역', '출간일', '쪽수', '판매가'])
df.head()

,제목,저자,번역,출간일,쪽수,판매가
0,"STEM CookBook, 전기 및 하이브리드 자동차(3판)",Iqbal Husain,"임원식 , 김정윤 , 김기찬 , 강동우",2023-01-10,592,35000
1,"IT CookBook, 컴퓨터 비전과 딥러닝",오일석,-,2023-01-05,664,39000
2,리더십 : 이론과 실제 (9판),Peter G. Northouse,김남현,2023-01-02,660,36000
3,"IT CookBook, JAVA 마스터",송미영,-,2023-01-01,732,30000
4,"IT CookBook, 데이터 과학을 위한 파이썬 프로그래밍(2판)",최성철,-,2023-01-01,540,30000


In [36]:
df.to_csv('hanbitNewbook.csv', index=False)